In [1]:
import subprocess
import shlex

def run_command(command):
    print("Launch " + command)
    process = subprocess.Popen(shlex.split(command), stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    while True:
        output = process.stdout.readline().decode()
        if output == '' and process.poll() is not None:
            break
        if output:
            print (output.strip())
    rc = process.poll()
    return rc
import os

class cd:
    """Context manager for changing the current working directory"""
    def __init__(self, newPath):
        self.newPath = os.path.expanduser(newPath)

    def __enter__(self):
        self.savedPath = os.getcwd()
        os.chdir(self.newPath)

    def __exit__(self, etype, value, traceback):
        os.chdir(self.savedPath)

In [2]:
import datetime
from dateutil.tz import tzlocal
import pandas as pd
import psycopg2
import matplotlib.pyplot as plt 

def get_df_process(host):
    host="vm-vmonly-db-from1"
    port=5432
    login="bonitauser"
    password="migrationTestDb_18"
    dbname="bonitadb"
    conn = psycopg2.connect("host = '{0}' dbname = '{1}' user = '{2}' password = '{3}' port = {4}".
             format(host, dbname, login, password, port))
    sql = "select api_id.sourceobjectid as id, to_timestamp(api_begin.startdate::float/1000) as start, to_timestamp(api_end.enddate::float/1000) as end, api_end.enddate - api_begin.startdate as duration \n"\
    "from ( \n"\
    "	select distinct sourceobjectid \n"\
    "	from arch_process_instance api) as api_id \n"\
    "join arch_process_instance api_begin on api_id.sourceobjectid = api_begin.sourceobjectid and api_begin.stateid = 0 \n"\
    "left join arch_process_instance api_end on api_id.sourceobjectid = api_end.sourceobjectid and api_end.stateid = 6 \n"\
    "order by id"
    cur = conn.cursor()
    cur.execute(sql)
    df = pd.DataFrame(cur.fetchall(), columns=["id", "start", "end", "duration"])
    return df

/usr/local/lib/python3.5/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [ ]:
# initialize infra
command = './create_vm.sh'
res = run_command(command)

In [3]:
# 

now=datetime.datetime.now(tzlocal())
strnow = now.strftime("%Y%m%d%H%M%S")
migration_result_name = "xp1_migr_{0}".format(strnow)
path = "./results/{0}/".format(migration_result_name)
import pathlib
pathlib.Path(path).mkdir(parents=True, exist_ok=True) 
print ("Migration result name : " + migration_result_name)
with cd("./test_executor/faban/master/bin"):
    run_command("./shutdown.sh")
    run_command("./startup.sh")


Migration result name : xp1_migr_20180621140036
Launch ./shutdown.sh
Launch ./startup.sh
Starting Faban Server
Please point your browser to http://127.0.0.1:9980/


### Initialize swarm on infrastructure 

In [6]:
# initialize Swarm on infra & upload files
import csv
#generate agent conf
agent_conf_file = "config.txt"
agent_conf={}
agent_conf["tenant_name"] = "tenant1"
agent_conf["login"] = "user1"
agent_conf["pwd"] = "user1"
agent_conf["nb_agent"] = 50
agent_conf["nb_process"] = 0

with open(path+agent_conf_file, 'w') as csvfile:
    row = [agent_conf["tenant_name"], agent_conf["login"], agent_conf["pwd"], agent_conf["nb_agent"], agent_conf["nb_process"]]
    wr = csv.writer(csvfile, delimiter=';')
    wr.writerow(row)
with open("./config.txt", 'w') as csvfile:
    row = [agent_conf["tenant_name"], agent_conf["login"], agent_conf["pwd"], agent_conf["nb_agent"], agent_conf["nb_process"]]
    wr = csv.writer(csvfile, delimiter=';')
    wr.writerow(row)

command = './init_swarm.sh'
res = run_command(command)

if (res != 0):
    print("Fail.")



Launch ./init_swarm.sh
[ERROR]: No handlers could be found for logger
"msrestazure.azure_active_directory"
[DEPRECATION WARNING]: 'include' for playbook includes. You should use
'import_playbook' instead. This feature will be removed in version 2.8.
Deprecation warnings can be disabled by setting deprecation_warnings=False in
ansible.cfg.

PLAY [vmonly] ******************************************************************

TASK [Gathering Facts] *********************************************************
ok: [vm-vmonly-db-ref]
ok: [vm-vmonly-bpm-agent1]
ok: [vm-vmonly-db-from1]
ok: [vm-vmonly-bpms-from1]
ok: [vm-vmonly-bpms-to1]
ok: [vm-vmonly-db-to1]

TASK [include_vars] ************************************************************
ok: [vm-vmonly-bpm-agent1]
ok: [vm-vmonly-bpms-to1]
ok: [vm-vmonly-bpms-from1]
ok: [vm-vmonly-db-from1]
ok: [vm-vmonly-db-ref]
ok: [vm-vmonly-db-to1]

TASK [docker : Run the equivalent of "apt-get update".] ************************
ok: [vm-vmonly-db-from1]
ok: [v

changed: [localhost]

TASK [swarm-manager : get docker swarm manager token] **************************
changed: [localhost]

TASK [swarm-manager : get docker swarm worker token] ***************************
changed: [localhost]

TASK [swarm-manager : debug] ***************************************************
ok: [localhost] => {
"ansible_default_ipv4.address": "10.0.0.5"
}

TASK [swarm-manager : set_fact] ************************************************
ok: [localhost]

TASK [swarm-manager : create registry] *****************************************
changed: [localhost]

PLAY [vmonly] ******************************************************************

TASK [swarm-worker : get docker info] ******************************************
ok: [vm-vmonly-bpm-agent1]
ok: [vm-vmonly-db-ref]
ok: [vm-vmonly-db-from1]
ok: [vm-vmonly-bpms-to1]
ok: [vm-vmonly-bpms-from1]
ok: [vm-vmonly-db-to1]

TASK [swarm-worker : quit swarm cluster] ***************************************
changed: [vm-vmonly-bpms-to1

### Initialize result name, Start Faban and reset Docker

In [ ]:



run_command("./reset_docker.sh")
    

### Provision and test for various numbers of processes

In [7]:
# provision and test


#dur = 30
faban_agent = 5

for i in range(1,21):
    name_migr = path + "{0}_migr_{1}".format(migration_result_name, faban_agent)
    dur = i * 30
    import os.path
    if not(os.path.exists("{0}.csv".format(name_migr))):
        file = open("{0}.csv".format(name_migr), "w")
        file.write("ts,nb,duration_disable,duration_move,duration_enable,duration_total\n")
        file.close()
    
    #run_command("./launch_agents.sh http://vm-vmonly-bpms-from1:8080/bonita agent-pre")
    tries = 3
    while tries >= 0:
        run_command("./reset_docker.sh")
        
        with cd ("./test_executor/templates/deployment_descriptor/template"):
            res = run_command("docker stack deploy --compose-file=docker-compose.yml {0}".format(migration_result_name))
            if (res != 0):
                tries = tries - 1
                import time
                time.sleep(10) 
                continue
            else:
                import time
                time.sleep(120) 
                break

    with cd ("./test_executor"):
        run_command("./benchflow_onlytest.sh generate_test {0} TestHumanTask-1.0.bar 5 {1} 0 tenant1".format(faban_agent, dur))

        run_command("./benchflow_onlytest.sh execute_tests {0}".format(migration_result_name))
    #run_command("docker service rm {0}_bpm-agent".format(migration_result_name))
    for i in range (1,7):
        res1 = run_command("./migrate.sh tenant1 {0}".format(name_migr))
        
        res1 = run_command("./migrate2.sh tenant1 {0}".format(name_migr))
        
    if (res1 !=0):
        print("Error in first migration.. skipping")
    #df_pre = get_df_process("vm-vmonly-db-from1")
    #df_pre.plot(x="start", y="duration", style=".")

    #date_postmigr = datetime.datetime.now(tzlocal())

    #run_command("./launch_agents.sh http://vm-vmonly-bpms-to1:8081/bonita agent-post")
    #with cd ("./test_executor"):
    #    run_command("./benchflow_onlytest.sh generate_test {0} TestHumanTask-1.0.bar 5 {1} 0 tenant1 config2.properties".format(faban_agent, dur))

    #    run_command("./benchflow_onlytest.sh execute_tests {0} config2.properties".format(migration_result_name))

    #df = get_df_process("vm-vmonly-db-from1")

    #df_pre = df[df["start"] < date_postmigr]
    #df_post = df[df["start"] >= date_postmigr]

    #df_pre.to_csv(path+"{0}_pre_{1}.csv".format(migration_result_name ,i))
    #df_post.to_csv(path+"{0}_post_{1}.csv".format(migration_result_name ,i))    

    #df_pre["start"] = df_pre["start"] - min(df_pre["start"])
    #df_post["start"] = df_post["start"] - min(df_post["start"])

    #df_pre = df_pre.set_index(pd.DatetimeIndex(df_pre["start"]))   
    #df_post=df_post.set_index(pd.DatetimeIndex(df_post["start"]))
    
    #fig,ax = plt.subplots()
    #df_pre.resample("1s", how="mean").plot(y="duration", style="ro",ax=ax)
    #df_post.resample("1s", how="mean").plot( y="duration", style="bx",ax=ax)
    
    #fig,ax = plt.subplots()
    #df_pre.plot(y="duration", style="ro",ax=ax)
    #df_post.plot( y="duration", style="bx",ax=ax)

Launch ./reset_docker.sh
"docker stack rm" requires at least 1 argument.
See 'docker stack rm --help'.

Usage:  docker stack rm STACK [STACK...] [flags]

Remove one or more stacks
Launch docker stack deploy --compose-file=docker-compose.yml xp1_migr_20180621140036
Ignoring unsupported options: restart

Creating network xp1_migr_20180621140036_default
Creating service xp1_migr_20180621140036_db-ref
Creating service xp1_migr_20180621140036_db-from
Creating service xp1_migr_20180621140036_bonita-from
Creating service xp1_migr_20180621140036_db-to
Creating service xp1_migr_20180621140036_bonita-to
Creating service xp1_migr_20180621140036_start_dependencies
Launch ./benchflow_onlytest.sh generate_test 5 TestHumanTask-1.0.bar 5 30 0 tenant1
The generated test ID is s74kGifsoqnHvDWBP78y2YXNaMRINOPd
Compiling the Faban driver
Buildfile: /home/ubuntu/scripts_gr/test_executor/drivers/s74kGifsoqnHvDWBP78y2YXNaMRINOPd/build.xml

init:
[mkdir] Created dir: /home/ubuntu/scripts_gr/test_executor/driv

insert into psql_target.flownode_instance select * FROM public.flownode_instance WHERE tenantid = 2;
insert into psql_target.process_instance select * FROM public.process_instance WHERE tenantid = 2;
insert into psql_target.waiting_event select * FROM public.waiting_event WHERE tenantid = 2;
insert into psql_target.event_trigger_instance select * FROM public.event_trigger_instance WHERE tenantid = 2;
insert into psql_target.pending_mapping select * FROM public.pending_mapping WHERE tenantid = 2;
insert into psql_target.message_instance select * FROM public.message_instance WHERE tenantid = 2;
insert into psql_target.connector_instance select * FROM public.connector_instance WHERE tenantid = 2;
insert into psql_target.ref_biz_data_inst select * FROM public.ref_biz_data_inst WHERE tenantid = 2;
insert into psql_target.multi_biz_data select * FROM public.multi_biz_data WHERE tenantid = 2;
insert into psql_target.document_mapping select * FROM public.document_mapping WHERE tenantid = 2;
in

DELETE FROM psql_target.form_mapping WHERE tenantid = 2;
DELETE FROM psql_target.page_mapping WHERE tenantid = 2;
DELETE FROM psql_target.process_content WHERE tenantid = 2;
DELETE FROM psql_target.proc_parameter WHERE tenantid = 2;
insert into psql_target.proc_parameter select * FROM public.proc_parameter WHERE tenantid = 2;
insert into psql_target.process_content select * FROM public.process_content WHERE tenantid = 2;
insert into psql_target.page_mapping select * FROM public.page_mapping WHERE tenantid = 2;
insert into psql_target.form_mapping select * FROM public.form_mapping WHERE tenantid = 2;
insert into psql_target.theme select * FROM public.theme WHERE tenantid = 2;
insert into psql_target.job_desc select * FROM public.job_desc WHERE tenantid = 2;
insert into psql_target.job_param select * FROM public.job_param WHERE tenantid = 2;
insert into psql_target.job_log select * FROM public.job_log WHERE tenantid = 2;
insert into psql_target.profile select * FROM public.profile WHERE 

create server psql_target foreign data wrapper postgres_fdw options (host '10.0.1.7', port '5433', dbname 'bonitadb');
create user mapping for bonitauser server psql_target options(user 'bonitauser', password 'migrationTestDb_18');
drop schema if exists psql_target;
create schema if not exists psql_target;
import foreign schema public from server psql_target into psql_target;
DELETE FROM psql_target.contract_data WHERE tenantid = 2;
DELETE FROM psql_target.actormember WHERE tenantid = 2;
DELETE FROM psql_target.actor WHERE tenantid = 2;
DELETE FROM psql_target.processcategorymapping WHERE tenantid = 2;
DELETE FROM psql_target.category WHERE tenantid = 2;
DELETE FROM psql_target.process_comment WHERE tenantid = 2;
DELETE FROM psql_target.process_definition WHERE tenantid = 2;
DELETE FROM psql_target.document WHERE tenantid = 2;
DELETE FROM psql_target.document_mapping WHERE tenantid = 2;
DELETE FROM psql_target.multi_biz_data WHERE tenantid = 2;
DELETE FROM psql_target.ref_biz_data_inst

DELETE FROM public.group_ WHERE tenantid = 2;
DELETE FROM public.page WHERE tenantid = 2;
DELETE FROM public.profilemember WHERE tenantid = 2;
DELETE FROM public.profileentry WHERE tenantid = 2;
DELETE FROM public.profile WHERE tenantid = 2;
DELETE FROM public.job_log WHERE tenantid = 2;
DELETE FROM public.job_param WHERE tenantid = 2;
DELETE FROM public.job_desc WHERE tenantid = 2;
DELETE FROM public.theme WHERE tenantid = 2;
DELETE FROM public.form_mapping WHERE tenantid = 2;
DELETE FROM public.page_mapping WHERE tenantid = 2;
DELETE FROM public.process_content WHERE tenantid = 2;
DELETE FROM public.proc_parameter WHERE tenantid = 2;
2018-06-21 14:20:28.399456 - activating tenant 2 on http://vm-vmonly-bpms-to1:8081/bonita
Tenant 2 is now ACTIVATED
2018-06-21 14:20:30.324525 - get number of process instances 2 on http://vm-vmonly-bpms-to1:8081/bonita
2018-06-21 14:20:30.337712 - done in 3.52350211143 s for 0 processes
Launch ./migrate2.sh tenant1 ./results/xp1_migr_20180621140036/xp1_

...

insert into psql_target.flownode_instance select * FROM public.flownode_instance WHERE tenantid = 2;
insert into psql_target.process_instance select * FROM public.process_instance WHERE tenantid = 2;
insert into psql_target.waiting_event select * FROM public.waiting_event WHERE tenantid = 2;
insert into psql_target.event_trigger_instance select * FROM public.event_trigger_instance WHERE tenantid = 2;
insert into psql_target.pending_mapping select * FROM public.pending_mapping WHERE tenantid = 2;
insert into psql_target.message_instance select * FROM public.message_instance WHERE tenantid = 2;
insert into psql_target.connector_instance select * FROM public.connector_instance WHERE tenantid = 2;
insert into psql_target.ref_biz_data_inst select * FROM public.ref_biz_data_inst WHERE tenantid = 2;
insert into psql_target.multi_biz_data select * FROM public.multi_biz_data WHERE tenantid = 2;
insert into psql_target.document_mapping select * FROM public.document_mapping WHERE tenantid = 2;
in

DELETE FROM psql_target.form_mapping WHERE tenantid = 2;
DELETE FROM psql_target.page_mapping WHERE tenantid = 2;
DELETE FROM psql_target.process_content WHERE tenantid = 2;
DELETE FROM psql_target.proc_parameter WHERE tenantid = 2;
insert into psql_target.proc_parameter select * FROM public.proc_parameter WHERE tenantid = 2;
insert into psql_target.process_content select * FROM public.process_content WHERE tenantid = 2;
insert into psql_target.page_mapping select * FROM public.page_mapping WHERE tenantid = 2;
insert into psql_target.form_mapping select * FROM public.form_mapping WHERE tenantid = 2;
insert into psql_target.theme select * FROM public.theme WHERE tenantid = 2;
insert into psql_target.job_desc select * FROM public.job_desc WHERE tenantid = 2;
insert into psql_target.job_param select * FROM public.job_param WHERE tenantid = 2;
insert into psql_target.job_log select * FROM public.job_log WHERE tenantid = 2;
insert into psql_target.profile select * FROM public.profile WHERE 

create server psql_target foreign data wrapper postgres_fdw options (host '10.0.1.5', port '5432', dbname 'bonitadb');
create user mapping for bonitauser server psql_target options(user 'bonitauser', password 'migrationTestDb_18');
drop schema if exists psql_target;
create schema if not exists psql_target;
import foreign schema public from server psql_target into psql_target;
DELETE FROM psql_target.contract_data WHERE tenantid = 2;
DELETE FROM psql_target.actormember WHERE tenantid = 2;
DELETE FROM psql_target.actor WHERE tenantid = 2;
DELETE FROM psql_target.processcategorymapping WHERE tenantid = 2;
DELETE FROM psql_target.category WHERE tenantid = 2;
DELETE FROM psql_target.process_comment WHERE tenantid = 2;
DELETE FROM psql_target.process_definition WHERE tenantid = 2;
DELETE FROM psql_target.document WHERE tenantid = 2;
DELETE FROM psql_target.document_mapping WHERE tenantid = 2;
DELETE FROM psql_target.multi_biz_data WHERE tenantid = 2;
DELETE FROM psql_target.ref_biz_data_inst

DELETE FROM public.user_ WHERE tenantid = 2;
DELETE FROM public.user_contactinfo WHERE tenantid = 2;
DELETE FROM public.role WHERE tenantid = 2;
DELETE FROM public.group_ WHERE tenantid = 2;
DELETE FROM public.page WHERE tenantid = 2;
DELETE FROM public.profilemember WHERE tenantid = 2;
DELETE FROM public.profileentry WHERE tenantid = 2;
DELETE FROM public.profile WHERE tenantid = 2;
DELETE FROM public.job_log WHERE tenantid = 2;
DELETE FROM public.job_param WHERE tenantid = 2;
DELETE FROM public.job_desc WHERE tenantid = 2;
DELETE FROM public.theme WHERE tenantid = 2;
DELETE FROM public.form_mapping WHERE tenantid = 2;
DELETE FROM public.page_mapping WHERE tenantid = 2;
DELETE FROM public.process_content WHERE tenantid = 2;
DELETE FROM public.proc_parameter WHERE tenantid = 2;
2018-06-21 14:21:12.947853 - activating tenant 2 on http://vm-vmonly-bpms-from1:8080/bonita
Tenant 2 is now ACTIVATED
2018-06-21 14:21:14.187775 - get number of process instances 2 on http://vm-vmonly-bpms-from1

KeyboardInterrupt: 

In [ ]:
run_command("./reset_docker.sh")
tries = 3
while tries >= 0:
    run_command("./reset_docker.sh")

    with cd ("./test_executor/templates/deployment_descriptor/template"):
        res = run_command("docker stack deploy --compose-file=docker-compose.yml {0}".format(migration_result_name))
        if (res != 0):
            tries = tries - 1
            import time
            time.sleep(10) 
            continue
        else:
            import time
            time.sleep(120) 
            break

for m in range(1,7):
    res1 = run_command("./migrate.sh tenant1 {0} true".format(name_migr))
    if (res1 != 0):
        print("Error in first migration.. skipping")
        continue
    res2 = run_command("./migrate2.sh tenant1 {0} true".format(name_migr))
    if (res2 != 0):
        print("Error in second migration.. ")

In [ ]:
df_pre